# Quality Assessment for ERA5 Drought Indicator: Reproducibility

Production date: 2026-xx-xx

**Please note that this repository is used for development and review, so quality assessments should be considered work in progress until they are merged into the main branch.**

Dataset version: 1.0.

Produced by: Enis Gerxhalija, Olivier Burggraaff (National Physical Laboratory).

## 🌍 Use case: Retrieving drought indicators from the ERA5-Drought dataset 

## ❓ Quality assessment question
* **Are the drought indicators in the ERA5-Drought dataset consistent with and reproducible from ERA5 data?**
* **Are the drought indicators in the ERA5-Drought dataset presented in a format that ensures optimal usability for users?**

Human-induced climate change is likely the primary driver behind the number of increased droughts and heavy precipitation since the 1950s, per the latest assessment report by the Intergovernmental Panel on Climate Change [IPCC, 2013](https://www.ipcc.ch/report/ar6/syr/downloads/report/IPCC_AR6_SYR_LongerReport.pdf). With further global warming at 1.5°C and above, heavy precipitation, flooding and drought events are projected to intensify and become more frequent in most regions of Africa, Asia, North America and Europe. The environmental and societal impact of such extreme weather events are far-reaching. In the United Kingdom alone, the 2020s have seen three of the five worst harvests on record, with extreme heat and drought in 2025 causing more than £800mn lost revenue in harvest, [Energy & Climate Intelligence Unit](https://mcusercontent.com/8ed7ad7972fae058e8f4fb7e8/files/6d02e6e7-8639-a44d-5a8c-2313124ef699/Costs_of_climate_analysis_011225.pdf). In 2023-2024, the Amazon region in Brazil faced an 18-month drought considered the most severe since drought monitoring began in 1954. By November 2024, it left 720 health centres in drought-affected areas of Brazil to become non-operational [UNICEF, 2024](https://www.unicef.org/media/165191/file/LACR-Flash-Update-11-November-2024.pdf).

A large scientific effort has gone into identifying areas more prone to drought along with monitoring areas currently experiencing drought conditions and accurately quantifying their severity [reference]. The objective quantification of drought severity remains an ongoing endeavour amongst scientists as there is not one physical variable that describes a drought. One might assume that drought severity can be measured by the total precipitation in that region, but this would overlook water-loss from the land surface through evapotranspiration, soil moisture levels, temperature anomalies and other natural variables. There is however consensus that drought indices, proxies based on long-term and shorter-term historical weather data, can accurately quantify drought severity and their impact, with studies linking the variability of drought indices to crop yields, [Vicente‐Serrano et al.](https://doi.org/10.1080/01431160500296032), and frequency of wildfires [reference]. Two widely-employed drought indices are the Standardised Precipitation Index (SPI) [reference](), endorsed by the World Metereological Organisation (WMO), and the more recent Standardised Precipitation-Evapotranspiration Index (SPEI) [reference](). 

The ERA5-Drought dataset provides a reanalysis-based dataset of the aforementioned indices using the ECMWF Reanalysis version 5 (ERA5), at a resolution of 0.25° globally (around 28 km) from the start of the reanalysis (in 1940) to today, [Keune et al., ‘ERA5–Drought’](https://doi.org/10.1038/s41597-025-04896-y). The ERA5-Drought dataset consists of 1 deterministic and 10 ensemble drought-index members from slightly different initial conditions, enabling an estimate of the uncertainty.

This notebook aims to give users much-needed confidence and transparency in the calculation of the two drought indices along with their quality flags. The C3S ERA5-Drought dataset must be consistent with and reproducible from its origins. Here, we assess this consistency and reproducibility by comparing drought-indicators retrieved from the ERA5-Drought dataset with their equivalents calculated from the origin dataset (or similar). While a full analysis and reproduction of every record within the C3S ERA5-Drought dataset is outside the scope of quality assessment (as it would require high-performance computing infrastructure), a case study with a narrower scope probes these quality attributes of the dataset and can be a jumping-off point for further analysis by the reader.

## 📢 Quality assessment statement

```{admonition} These are the key outcomes of this assessment
:class: note
* Conclusion 1
* Conclusion 2

## 📋 Methodology

This quality assessment tests the consistency between drought indices retrieved from the [C3S ERA5-Drought dataset] and their equivalents calculated from the origin datasets, as well as the reproducibility and usability of said dataset.

We will examine the SPI and SPEI drought indicators calculated from the following datasets:

(include table here of the parameter, description of that parameters, and the origin dataset)

The analysis and results are organised in the following steps, which are detailed in the sections below:

**[](section-codesetup)**
 * Import all required libraries.
 * Define helper functions.
 * \\\\\\\\\\\\\\\\\\\\\\

**[](section-2)**
 * Define SPI Indicator
 * Download ERA5 precipitation
 * Accumulate
 * Calculate SPI
 * Calculate quality flags from ERA5 data
 * Download quality flags from ERA5-Drought
 * Compare quality flags
 * Download ERA5-Drought SPI
 * Comparison
 * \\\\\\\\\\\\\\\\\\

**[](section-3)**
 * Define SPEI Indicator
 * Download ERA5 potential evaporation
 * Accumulate
 * Calculate SPEI
 * Calculate quality flags from ERA5 data
 * Download quality flags from ERA5-Drought
 * Compare quality flags
 * Download ERA5-Drought SPEi
 * Comparison
 * \\\\\\\\\\\\\\\\\\\\\

**[](section-4)**
 
**[](section-5)** 

Any further notes on the method could go here (explanations, caveats or limitations).

## 📈 Analysis and results

(section-codesetup)=
### 1. Code setup
```{note}
This notebook uses [earthkit](https://github.com/ecmwf/earthkit) for downloading ([earthkit-data](https://github.com/ecmwf/earthkit-data)) and visualising ([earthkit-plots](https://github.com/ecmwf/earthkit-plots)) data. Because earthkit is in active development, some functionality may change after this notebook is published. If any part of the code stops functioning, please raise an issue on our GitHub repository so it can be fixed.
```

#### Import required libraries
In this section, we import all the relevant packages needed for running the notebook.

In [ ]:
# Input / Output
from pathlib import Path
import earthkit.data as ekd
import warnings

# General data handling
import numpy as np
import pandas as pd
import xarray as xr
from functools import partial

# Analysis
import calendar
from scipy import stats

# Visualisation
import earthkit.plots as ekp
from earthkit.plots.styles import Style
import matplotlib.pyplot as plt
plt.rcParams["grid.linestyle"] = "--"
from tqdm import tqdm  # Progress bars

# Visualisation in Jupyter book -- automatically ignored otherwise
try:
    from myst_nb import glue
except ImportError:
    glue = None

# Type hints
from typing import Callable, Iterable, Optional
from scipy.stats import rv_continuous as Distribution
from earthkit.plots.geo.domains import Domain
AnyDomain = (Domain | str)

#### Helper functions
This section defines some functions and variables used in the following analysis, allowing code cells in later sections to be shorter and ensuring consistency.

##### Data (pre-)processing
The following functions handle [data chunking in dask](https://docs.xarray.dev/en/latest/user-guide/dask.html) for computational efficiency:

In [ ]:
def rechunk(data: xr.Dataset) -> xr.Dataset:
    """ Re-chunk a dataset into pre-determined optimal chunks. """
    # Might need to be adjusted for different coordinate names
    return data.chunk({"valid_time": -1, "latitude": 103, "longitude": 360})

##### Accumulation periods
The following cells contain constants and functions used in accumulating variables (e.g. precipitation) over time:

In [ ]:
# Define constants such as the accumulation periods to use
ACCUMULATION_PERIODS = [1, 3, 6, 12, 24, 36, 48]  # Months
MONTHS = range(1, 13)  # January to December (inclusive)

# Perform accumulation
def accum_var(data: xr.Dataset, var: str, *,
              accumulation_periods: Iterable[int]=ACCUMULATION_PERIODS, time_dim: Optional[str]=None) -> xr.Dataset:
    """
    Compute the precipitation / potential evaporation accumulation window. 
    
    1. Convert precipitation/potential evaporation from meters to millimeters.
    2. Compute monthly totals (accounting for days in month).
    3. Add rolling accumulation windows on monthly totals.
    """
    # Detect time dimension if not provided
    # TO DO: Use the first dimension that contains "time"?
    if time_dim is None:
        if 'valid_time' in data.dims:
            time_dim = 'valid_time'
        elif 'forecast_reference_time' in data.dims:
            time_dim = 'forecast_reference_time'
        elif "time" in data.dims:
            time_dim = "time"
        else:
            raise ValueError("No valid time dimension found. Expected 'valid_time' or 'forecast_reference_time'.")

    # Ensure time is sorted
    # TO DO: Is this necessary?
    data = data.sortby(time_dim)
    
    # Step 1: Convert to mm
    data[f'{var}_mm'] = data[var] * 1000

    # Step 2: Compute monthly totals
    # TO DO: Is this necessary for the analysis (I don't think so) or only for the plot?
    # In the latter case, you could (don't need to) either make this optional or split it out altogether
    time_index = pd.to_datetime(data[time_dim].values)
    
    days_in_month = xr.DataArray(
        time_index.days_in_month,
        coords={time_dim: data[time_dim]},
        dims=[time_dim]
    )

    data[f'{var}_mm_monthly_total'] = data[f'{var}_mm'] * days_in_month
    data[f'{var}_mm_monthly_total'] = data[f'{var}_mm_monthly_total'].astype("float64")

    # Step 3: Add rolling accumulation windows
    for period in accumulation_periods:
        rolling_sum = data[f"{var}_mm_monthly_total"].rolling({time_dim: period}, center=False).sum()
        data[f'{var}_mm_accum_{period}m'] = rolling_sum

    data = data.chunk({time_dim: -1})

    return data.drop_vars([f"{var}", f"{var}_mm", f"{var}_mm_monthly_total"])

##### Calculating SPI / SPEI
The following cells contain code for calculating SPI and SPEI.

The following functions ... fitting:

In [ ]:
def fit_monthly_distributions_xr(reference_data: xr.Dataset, var: str, *,
                                 accumulation_periods: Iterable[int]=ACCUMULATION_PERIODS, time_dim: Optional[str]=None) -> xr.Dataset:
    """
    Fit gamma/genlogistic distributions for each month and accumulation period using xarray.
    Data are assumed to have been sliced to the reference period.
    """
    monthly_params = {}

    # Detect time dimension if not provided
    # TO DO: Use the first dimension that contains "time"?
    # TO DO: Refactor so can be re-used
    if time_dim is None:
        if 'valid_time' in reference_data.dims:
            time_dim = 'valid_time'
        elif 'forecast_reference_time' in reference_data.dims:
            time_dim = 'forecast_reference_time'
        else:
            raise ValueError("No valid time dimension found. Expected 'valid_time' or 'forecast_reference_time'.")

    for period in accumulation_periods:
        var_name = f'{var}_mm_accum_{period}m'
        for month in range(1, 13):
            # Select month subset
            month_subset = reference_data[var_name].where(reference_data[time_dim].dt.month == month, drop=True)

            # Drop NaNs and flatten
            values = month_subset.values.flatten()
            values = values[~np.isnan(values)]
            if len(values) > 0:
                # TO DO: Make the fitting distribution an argument to the function,
                # then use functools.partial to create defaults
                # e.g. function call: (..., var: str, distribution: Callable, ...)
                # and the part below becomes alpha, loc, beta = distribution.fit(values)
                # and then define
                # fit_monthly_spi  = partial(fit_monthly_distributions_xr, distribution=stats.gamma)
                # fit_monthly_spei = partial(fit_monthly_distributions_xr, distribution=stats.genlogistic)
                if var == "tp":
                    alpha, loc, beta = stats.gamma.fit(values)
                elif var == "pev" or "wb":
                    alpha, loc, beta = stats.genlogistic.fit(values) # alpha is shape parameters, loc = location, beta is scale.

            # Else?
            monthly_params[(month, period)] = (alpha, loc, beta) # TODO: put into a xarray.
    return monthly_params

In [ ]:
# General fitting function
def fit_monthly_distributions_new(reference_data: xr.Dataset, distribution: Distribution, *,
                                  time_dim: Optional[str]=None) -> xr.Dataset:
    """
    Fit distributions (e.g. gamma) for each month and accumulation period using xarray parallelisation.
    Data are assumed to have been sliced to the reference period.
    """
    # Define fitting function
    def fit(y):
        y = y[np.isfinite(y)]
        params = distribution.fit(y) #500 ,loc=-1000, scale = 5)
        params = np.stack(params, axis=-1)  # Extend with axis for stats (alpha, loc, scale ...)
        return params

    # Split dataset by month
    if time_dim is None:  # Detect time dimension if not provided
        time_dim = next(dim for dim in reference_data.dims if "time" in dim)  # TO DO: Refactor so can be re-used
    reference_data_by_month = reference_data.groupby(reference_data[time_dim].dt.month)

    # Apply fitting function by month
    params = xr.apply_ufunc(fit, reference_data_by_month,
                            input_core_dims=[[time_dim]], output_core_dims=[["stat"]],
                            vectorize=True,
                            dask="parallelized",
                            dask_gufunc_kwargs={"output_sizes": {"stat": distribution.numargs+2}},  # e.g. 3 for gamma (alpha, loc, scale)
                            output_dtypes=[np.float64],
                           )
    params = params.chunk({"month": -1})

    return params

# Fitting functions for SPI, SPEI specifically
fit_monthly_spi  = partial(fit_monthly_distributions_new, distribution=stats.gamma) # partial function
fit_monthly_spei = partial(fit_monthly_distributions_new, distribution=stats.genlogistic)

The following functions ... applying fitted distributions to calculate SPI / SPEI:

In [ ]:
# General computing function
def compute_monthly_series_new(data: xr.Dataset, monthly_params: xr.Dataset, distribution: Distribution, *,
                               index_name: Optional[str]=None, time_dim: Optional[str]=None) -> tuple[xr.Dataset]:
    """
    Compute SPI time series for each accumulation period using fitted distribution parameters.
    Returns two datasets: CDF values and index (SPI/SPEI) values.
    """
    # Create a month dimension for broadcasting with the one in monthly_params
    if time_dim is None:  # Detect time dimension if not provided
        time_dim = next(dim for dim in data.dims if "time" in dim)  # TO DO: Refactor so can be re-used
    month_da = data[time_dim].dt.month.rename("month")

    params_fitted = monthly_params
    
    # Extract parameters
    nr_params = params_fitted.sizes["stat"]  # 3 for gamma and genlogistic
    params_extracted = [params_fitted.sel(stat=j).sel(month=month_da) for j in range(nr_params)]

    # Calculate CDF values by month
    cdf = xr.apply_ufunc(distribution.cdf, data, *params_extracted,
                         input_core_dims=[[], [], [], []], output_core_dims=[[]],
                         vectorize=True, dask="parallelized",
                         output_dtypes=[np.float64],
                         keep_attrs=True
                        )
    cdf = cdf.chunk({time_dim: -1})
    
    clipped = cdf.clip(1e-16, 1 - 1e-16)

    # Convert CDF to index
    index = xr.apply_ufunc(stats.norm.ppf, clipped,
                           input_core_dims=[[]], output_core_dims=[[]],
                           vectorize=True, dask="parallelized",
                           output_dtypes=[np.float64],
                           keep_attrs=True
                          )

    # Optional: Rename variables in index dataset to e.g. SPI12
    if index_name:
        accumulation_variables = {var: var.split("_")[-1][:-1] for var in index.variables if "accum" in var}  # Get periods as number strings
        rename_variables = {var: f"{index_name}{accumulation_period}" for var, accumulation_period in accumulation_variables.items()}
        index = index.rename_vars(rename_variables)

    return cdf, index

# Computing functions for SPI, SPEI specifically
compute_monthly_spi  = partial(compute_monthly_series_new, distribution=stats.gamma,       index_name="SPI")
compute_monthly_spei = partial(compute_monthly_series_new, distribution=stats.genlogistic, index_name="SPEI")

def compute_spi_dataset(data, accum_periods, n_points = 1000):
    """
    Fit gamma distribution for each month and accumulation period,
    then compute SPI, CDF, and PDF for a linspace of precipitation values.
    """
    result_vars = {}

    for period in accum_periods:
        var_name = f'tp_mm_accum_{period}m'
        
        tp_values = data[var_name]

        # Prepare storage lists
        spi_list, cdf_list, pdf_list, precip_list = [], [], [], []

        for month in range(1, 13):
            # Select month subset
            month_data = tp_values.where(tp_values['valid_time.month'] == month, drop=True).values
            month_data = month_data[month_data > 0]                
            month_data = month_data[~np.isnan(month_data)]

                
            # Fit gamma distribution
            alpha, loc, beta = stats.gamma.fit(month_data, floc=0)

            vals = np.linspace(month_data.min(), month_data.max(), n_points)

            # Compute PDF, CDF, SPI
            cdf_vals = stats.gamma.cdf(vals, a=alpha, loc=loc, scale=beta)
            spi_vals = stats.norm.ppf(cdf_vals)
            pdf_vals = stats.gamma.pdf(vals, a=alpha, loc=loc, scale=beta)

            # Append to lists
            precip_list.append(vals)
            spi_list.append(spi_vals)
            cdf_list.append(cdf_vals)
            pdf_list.append(pdf_vals)

            
        # Convert lists to DataArrays
        months = np.arange(1, 13)
        
        result_vars[f'precip_{period}m'] = xr.DataArray(precip_list, dims=['month', 'value_index'], coords={'month': months})
        result_vars[f'SPI_{period}m'] = xr.DataArray(spi_list, dims=['month', 'value_index'], coords={'month': months})
        result_vars[f'CDF_{period}m'] = xr.DataArray(cdf_list, dims=['month', 'value_index'], coords={'month': months})
        result_vars[f'PDF_{period}m'] = xr.DataArray(pdf_list, dims=['month', 'value_index'], coords={'month': months})

    return xr.Dataset(result_vars)
    
def zero_precip_monthly_xr(prec_data: xr.Dataset, cdf_ds: xr.Dataset, reference_window, accum_periods: Iterable[int]=ACCUMULATION_PERIODS):
    """
    Adjust CDF for zero precipitation probability in xarray.
    """
    # Slice reference period, find time variable, and months.
    reference_data = prec_data.sel(**reference_window)

    reference_data = reference_data.to_array("accumulation_period")
    
    time_dim = next(dim for dim in reference_data.dims if "time" in dim)  # TO DO: Refactor so can be re-used
    month = reference_data[time_dim].dt.month

    # Threshold precipitation
    prec_eps = 0.01

    # All months with precipitation less than precipitation.
    is_zero = (reference_data <= prec_eps)

    # Zero precipitation stats. 
    n_zero  = is_zero.groupby(month).sum(dim=time_dim)    
                                         
    # Count total months per calendar month
    n_month = reference_data.groupby(month).count(dim=time_dim)

    # Ratio of months with zero precipitation
    ratio_zero = n_zero / n_month

    # Weighted probability with zero precipitation.
    p_zero = xr.where(
        n_zero > 0,
        (n_zero + 1) / (2 * (n_month + 1)),
        0
    ) 

    cdf = cdf_ds.to_array("accumulation_period")
    
    cdf_data_by_month = cdf.groupby("valid_time.month") 

    adjusted_cdf = cdf_data_by_month.map(
        lambda x: p_zero.sel(month=x["valid_time.month"]) + (1 - p_zero.sel(month=x["valid_time.month"])) * x
    ) #  pass formula onto each month and then just pass back to adjusted_cdf
    
    # Convert back to Dataset
    adjusted_cdf = adjusted_cdf.to_dataset("accumulation_period")

    # Summary of stats
    stats_summary = xr.Dataset({
        "Zero-Precip Count": n_zero,
        "Total Months": n_month,
        "Prob Zero Precip": p_zero,
        "Historical Ratio": ratio_zero,
    })

    stats_summary = stats_summary.assign_coords(accumulation_period=("accumulation_period", accum_periods))
    
    return adjusted_cdf, stats_summary 
    
def xr_shapiro_test(spi_ds: xr.Dataset,accum_periods: Iterable[int]=ACCUMULATION_PERIODS, months = range(1,13) )  :
    # TODO: Optimise this function with dask and adapt to above.
    spi_ref = spi_ds.sel(valid_time=slice("1991-01-01", "2020-12-31"))

    spi_ref = spi_ref.chunk({'valid_time': -1})        # make valid_time one chunk
    
    spi_ref = spi_ref.where(np.isfinite(spi_ref)) # Mask non-finite values before masking.
    
    spi_ref_by_month = spi_ref.groupby("valid_time.month")
    
    # Perform shapiro on xarray
    stat, pval = xr.apply_ufunc(stats.shapiro, spi_ref_by_month, 
                           input_core_dims=[['valid_time']], output_core_dims=[[],[]],
                           vectorize=True, dask="parallelized",
                           output_dtypes=[np.float64, np.float64],
                           keep_attrs=True
                          )
    
    normality = xr.where(pval < 0.05, 0, 1) #  Values < 0.05 → 0
    
    return stat, pval, normality

def cdf_to_spi_transform(adjusted_cdf_ds):

    clipped = adjusted_cdf_ds.clip(1e-16, 1 - 1e-16)
    
    # Convert CDF to index
    adjusted_spi_ds = xr.apply_ufunc(stats.norm.ppf, clipped, 0.0, 1.0,  
                           input_core_dims=[[], [], []], output_core_dims=[[]],
                           vectorize=True, dask="parallelized",
                           output_dtypes=[np.float64],
                           keep_attrs=True
    )
    
    accumulation_variables = {var: var.split("_")[-1][:-1] for var in adjusted_spi_ds.variables if "accum" in var}  # Get periods as number strings
    rename_variables = {var: f"{"SPI"}{accumulation_period}" for var, accumulation_period in accumulation_variables.items()}
    
    return adjusted_spi_ds.rename_vars(rename_variables)

##### Helper functions for reading data.

In [ ]:
def era5_drought_api_multiple(indicator, var, accum_period = [1, 3, 6, 12, 24, 36, 48]):
    ind = indicator.strip().upper()
    if ind not in {"SPI", "SPEI"}:
        raise ValueError("indicator must be 'SPI' or 'SPEI'")
    ind = ind.lower()

    var_key = var.strip().lower()
    if var_key not in {"prob_zero", "quality"}:
        raise ValueError("var must be 'prob_zero' or 'quality'")
    if var_key == "prob_zero":
        dataset_name = "derived-drought-historical-monthly"
        request_var = f"probability_of_zero_precipitation_{ind}"
        source_var_name = "pzero"
        rename_prefix = "prob_zero"
    else:
        dataset_name = "derived-drought-historical-monthly"
        request_var = f"test_for_normality_{ind}"
        source_var_name = "significance"
        rename_prefix = "significance"
    

    out = []
    
    for p in accum_period:
        request = {
            "variable": [request_var],
            "accumulation_period": [str(p)],
            "version": "1_0",
            "product_type": ["reanalysis"],
            "dataset_type": "consolidated_dataset",
            "month": [f"{m:02d}" for m in range(1, 13)],
        }

        ds = ekd.from_source("cds", dataset_name, request).to_xarray(compat="equals")
        
        if source_var_name not in ds.variables:
            raise KeyError(
                f"Expected variable '{source_var_name}' not found for period {p}. "
                f"Available: {list(ds.variables)}"
            )

        new_name = f"{rename_prefix}_{p}"
        ds_renamed = ds.rename({source_var_name: new_name})[[new_name]]
        out.append(ds_renamed)

    out = xr.merge(out, compat="override")
    out.to_array("accumulation_period")
    out.assign_coords(accumulation_period=("accumulation_period", ACCUMULATION_PERIODS))
        
    return out

(section-2)=
### 2. Calculate SPI at one location from ERA5 reanalysis data

#### Download monthly precipitation data 
First, we download the monthly-mean total precipitation data from the ERA5 reanalysis.
Generally,
one would use the [_ERA5 monthly averaged data on single levels from 1940 to present_ (reanalysis-era5-single-levels-monthly-means)](https://doi.org/10.24381/cds.f17050d7) dataset for this,
which provides pre-calculated monthly means at 0.25° by 0.25° resolution.
For this assessment,
to be as close to the ERA5-Drought data processing pipeline as possible
and
to make use of some of MARS's functionalities (see [below](section-4)),
we instead use the [_Complete ERA5 global atmospheric reanalysis_ (reanalysis-era5-complete)](https://doi.org/10.24381/cds.143582cf) dataset.

In this assessment,
we will calculate SPI and SPEI for each month
(with different accumulation periods, see below)
for the years 1940–2024.
For the reference period,
we will use the World Meteorological Organization (WMO) current standard 30-year reference period of 1991–2020,
which is also used in ERA5-Drought.
Both of these date ranges can be adjusted in the cell below when running the analysis yourself:

In [ ]:
# Define your preferred analysis and reference periods
years           = (1940, 2024)  # Years for the analysis (inclusive)
years_reference = (1991, 2020)  # Years for the reference period (inclusive)

# Derived variables for convenience:
reference_window = {"valid_time": slice(f"{years_reference[0]}-01-01", f"{years_reference[1]}-12-01"),}  #  Slice (1991-01-01, 2020-12-01)
entire_window = {"valid_time": slice(f"{years[0]}-01-01", f"{years[1]}-12-01"),}  #  Slice (1940-01-01, 2024-12-01)

Having defined our target years, we can now define our CDS request.
First, we define a template with some default parameters
(e.g. years, data format)
that will also be used later in the notebook.
Additional information for specific downloads
(e.g. variable, data stream)
is mixed into this template where relevant.

This notebook uses [earthkit-data](https://github.com/ecmwf/earthkit-data) to download files from the CDS.
If you intend to run this notebook multiple times, it is highly recommended that you [enable caching](https://earthkit-data.readthedocs.io/en/latest/guide/caching.html) to prevent having to download the same files multiple times.
If you prefer not to use earthkit, the following requests can also be used with the [cdsapi module](https://cds.climate.copernicus.eu/how-to-api#linux-use-client-step).
In either case (earthkit-data or cdsapi), it is required to set up a CDS account and API key as explained [on the CDS website](https://cds.climate.copernicus.eu/how-to-api).

In [ ]:
ID_ERA5 = "reanalysis-era5-complete"

request_era5_template = {
    "class": "ea",            # Default for ERA5
    # Dates: ERA5 takes these in the format 19400101/19400201/.../20241101/20241201
    # The following line generates a string in said format from the chosen year range
    "date": "/".join(f"{year}{month:02}01"
            for year in range(years[0] ,years[1]+1)
            for month in MONTHS),
    "expver": "1",            # ERA5 consolidated data
    "levtype": "sfc",         # Surface
    "grid": "0.25/0.25",      # Grid: 0.25° by 0.25°
    "type": "fc",             # Forecast
    "data_format": "netcdf",  # NetCDF data
}

In this section, we want to download 
total precipitation data (variable `228.128`)
from the
`moda` stream (monthly-mean reanalysis data),
so we mix this information into the template
and submit the request to the CDS.
More information about formatting these requests is available in the [MARS ERA5 catalogue](https://apps.ecmwf.int/data-catalogues/era5/?class=ea).

In [ ]:
request_era5_precipitation_moda = {
    "param": "228.128",       # Variable: Total precipitation
    "stream": "moda",         # Data stream: Monthly mean reanalysis
} | request_era5_template

In [ ]:
# Download data and convert to desired format
era5_monthly_mean_reanal = ekd.from_source("cds", ID_ERA5, request_era5_precipitation_moda)  # Download as field list
era5_monthly_mean_reanal = era5_monthly_mean_reanal.to_xarray(compat="equals")  # Convert to xarray dataset
era5_monthly_mean_reanal = rechunk(era5_monthly_mean_reanal)  # Re-chunk for speed gain in fitting
era5_monthly_mean_reanal  # Display in notebook


#### 1. Calculate moving average for different accumulation periods.
A time series of precipitation for one grid point is extracted and precipitation is accumulated over the previous $n$ months using a moving window. Since the origin precipitation data from ERA5 is from the monthly mean dataset, we calculate the total precipitation for that month by multiplying with the total number of days in that month, correcting for leap years.

The different accumulation windows are used to determine the timescale of the drought. The longer the drought, typically the more severe the impact it will have.

* **1-, 3-month window**: useful for soil moisture, flow in small creaks.
* **6-, 12-month window**: looking at reservoir storage, reduced stream flow.
* **24-, 36-, 48-month window**: groundwater recharge, reduced reservoir. 


TO DO: Some text about the user choosing their point here, which point did we choose and why

In [ ]:
# Define your preferred site for the SPI example
example_site = {"latitude": 9.25, "longitude": 40.5,}

In [ ]:
# Select the grid point and time slice for Ethiopia
precipitation_example_site = era5_monthly_mean_reanal.sel(**example_site)

# Perform the accumulation for each accumulation period
precipitation_example_site = accum_var(precipitation_example_site, "tp")

# Display result
precipitation_example_site

In [ ]:
# TO DO: Refactor so the same function can be re-used for TP–PEV later
plt.figure(figsize=(12, 6))
for p in ACCUMULATION_PERIODS:
    var_name = f'tp_mm_accum_{p}m'
    plt.plot(precipitation_example_site['valid_time'].values, precipitation_example_site[var_name].values, label=f'{p}-month')

# Customize plot
plt.title(f"Precipitation Accumulation at ({example_site['latitude']} °N, {example_site['longitude']} °E)")
plt.xlabel("Time")
plt.ylabel("Precipitation [mm]")
plt.legend(title="Accumulation period", reverse=True) # plot legend in reverse to match order of the lines (top-to-bottom).
plt.grid(True)
plt.tight_layout()
plt.show()
plt.close()

#### 2. Fit gamma distribution (over calendar months) to different accumulation periods.

The gamma distribution [reference] is fitted only to the data within the reference period (1991-2020). A separate distribution is fitted for each calendar month per accumulation window. 

For e.g. : in the 3-month accumulation window, a gamma distribution is fitted on all 30 Januaries in that reference period, all the Februaries (30 of them) and so forth... 

This fitting is done with the [scipy.stats.gamma](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.gamma.html) object in Python. Three parameters are then outputted per calendar month, per accumulation period: the shape ($\alpha$), location ($\beta$) and scale ($\lambda$) parameter.

In [ ]:
# Temporarily moved into separate cell for testing -- merge back into fitting cell below when done
precipitation_example_site_reference = precipitation_example_site.sel(**reference_window) # select only reference precipitation window.
precipitation_example_site_entire = precipitation_example_site.sel(**entire_window) # select only reference precipitation window.

In [ ]:
# OB function testing (new)
params_fitted = fit_monthly_spi(precipitation_example_site_reference)
params_fitted 

#### 3. Compute SPI series

From the parameters from the gamma distribution fitting, we calculate the cumulative distribution function or CDF, $F(x, \alpha, \beta, \lambda)$, up to the accumulated precipitation $x$, from the probability distribution function or PDF, $f(u, \alpha, \beta, \lambda)$:

\begin{equation}
F(x, \alpha, \beta, \lambda) = \int_{0}^{x}f(u, \alpha, \beta, \lambda) \, du.
\end{equation}

A one-to-one mapping of SPI-index to accumulated precipitation value is then obtained by transforming the cumulative probability values to a standard normal distribution with a mean ($\mu$) of zero and standard deviation ($\sigma$) of 1. This mapping is applied to the historical record of the accumulated precipitation values in that calendar month and accumulation window.

\begin{equation}
\text{SPI-index} = \Phi^{-1}\big(F(x, \alpha, \beta, \lambda)\big)
\end{equation}


In [ ]:
# Compute SPI series -- new function
cdf_ds, spi_ds = compute_monthly_spi(precipitation_example_site_entire, params_fitted)

##### Plot of Precipitation vs CDF (for different accumulation windows).

In [ ]:
plt.figure(figsize=(12, 6))

from scipy.interpolate import CubicSpline

spi_month_ds = spi_ds_prec.where(spi_ds_prec.month == 5, drop=True)

for p in accum_periods:
    var1_name = f'SPI_{p}m'
    var2_name = f'CDF_{p}m'
    var3_name = f'precip_{p}m'
    cs = CubicSpline(spi_month_ds[var2_name].values.ravel(), spi_month_ds[var3_name].values.ravel())
    x_range = np.arange(-0.01, 1.01, 0.01)
    plt.plot(cs(x_range), x_range, label=f'{p}-month')
    # plt.scatter(spi_ds[var1_name].values, cdf_ds[var2_name].values, label=f'{p}-month')
    # plt.scatter(spi_month_ds[var1_name].values, spi_month_ds[var3_name].values, label=f'{p}-month', s=10)

# Customize plot
plt.title('CDF vs Precipitaiton')
plt.xlabel('Precipitation (mm)')
plt.ylabel('CDF')
plt.ylim([0,1])
plt.legend(title='Accumulation Period')
plt.grid(True)
plt.tight_layout()
plt.show()

##### Plot of Precipitation vs SPI (for different accumulation windows).

In [ ]:
plt.figure(figsize=(12, 6))

from scipy.interpolate import CubicSpline

spi_month_ds = spi_ds_prec.where(spi_ds_prec.month == 5, drop=True)

for p in accum_periods:
    var1_name = f'SPI_{p}m'
    var2_name = f'CDF_{p}m'
    var3_name = f'precip_{p}m'
    cs = CubicSpline(spi_month_ds[var1_name].values.ravel(), spi_month_ds[var3_name].values.ravel())
    x_range = np.arange(-3, 3, 0.01)
    plt.plot(x_range, cs(x_range), label=f'{p}-month')
    # plt.scatter(spi_ds[var1_name].values, cdf_ds[var2_name].values, label=f'{p}-month')
    # plt.scatter(spi_month_ds[var1_name].values, spi_month_ds[var3_name].values, label=f'{p}-month', s=10)

# Customize plot
plt.title('Precipitation vs SPI')
plt.xlabel('SPI')
plt.ylabel('Precipitation (mm)')
plt.legend(title='Accumulation Period')
# plt.xlim([-1, 1])
plt.grid(True)
plt.tight_layout()
plt.show()

##### Plot of SPI vs CDF (for different accumulation windows).

In [ ]:
plt.figure(figsize=(12, 6))

from scipy.interpolate import CubicSpline

spi_month_ds = spi_ds_prec.where(spi_ds_prec.month == 5, drop=True)

for p in accum_periods:
    var1_name = f'SPI_{p}m'
    var2_name = f'CDF_{p}m'
    var3_name = f'precip_{p}m'
    cs = CubicSpline(spi_month_ds[var1_name].values.ravel(), spi_month_ds[var2_name].values.ravel())
    x_range = np.arange(-3, 3, 0.01)
    plt.plot(x_range, cs(x_range), label=f'{p}-month')
    # plt.scatter(spi_ds[var1_name].values, cdf_ds[var2_name].values, label=f'{p}-month')
    # plt.scatter(spi_month_ds[var1_name].values, spi_month_ds[var3_name].values, label=f'{p}-month', s=10)

# Customize plot
plt.title('Precipitation vs SPI')
plt.xlabel('SPI')
plt.ylabel('Precipitation (mm)')
plt.legend(title='Accumulation Period')
# plt.xlim([-1, 1])
plt.grid(True)
plt.tight_layout()
plt.show()

#### 4. Calculating historical ratio of months without precipitation and finding zero adjusted SPI (ref dataset)
In regions of extremely low precipitation (e.g. the Sahara desert), months may have little to no accumulated precipitation. This poses a problem when fitting the gamma distribution since it is defined only for positive, real values. Furthermore, a criteria is set so that out of the 30 months in the reference period, per calendar month & accumulation window, 10 months must have non-zero accumulated precipitations. Otherwise, the calendar month is reported as having no value.

Months with zero precipitation are defined as less than 0.1 mm by the [Copernicus European (EDO) and Global (GDO) Drought Observatories](https://drought.emergency.copernicus.eu/). The ERA5-Drought dataset appears to define months with zero precipitation as having an accumulated precipitation exactly equal to zero. 

To get around some months having zero precipitation, the CDF is adjusted with the historical occurrence $p_{0}$ of periods with zero precipitation [Stagge et al., ‘Candidate Distributions for Climatological Drought Indices](https://doi.org/10.1002/joc.4267).

\begin{equation}
F_{p_{0}}(x_{p>0},  \alpha, \beta, \lambda) = p_{0} + \big(1 - p_{0}\big) \, F(x_{p>0},  \alpha, \beta, \lambda),
\end{equation}

where $F_{p_{0}}(x_{p>0},  \alpha, \beta, \lambda)$ is the CDF adjusted for zero precipitation.

This simple treatment can lead to a z-distribution that is skewed, with a non-zero mean. Therefore, special care must be taken, and $p_{0}$ must be adjusted for the "centre of probability mass" following:

\begin{equation}
\bar{p}_{0} = \frac{n_{p=0} + 1}{2(n + 1)},
\end{equation}

where $n_{p=0}$ are the number of calendar months in the reference period per accumulation window, and n are the total number of calendar months (30 reference calendar months).

\begin{equation}
F_{\bar{p}_{0}}(x, \alpha, \beta, \lambda) =
\begin{cases}
\bar{p}_{0} + (1 - \bar{p}_{0}) \, F(x_{p>0},  \alpha, \beta, \lambda), & x > 0, \\[6pt]
\frac{n_{p=0} + 1}{2(n+1)}, & x = 0.
\end{cases}
\end{equation}
This maintains the mean SPI value of zero, allowing for an objective statistical comparison.

In [ ]:
adjusted_cdf_ds, stats_sum = zero_precip_monthly_xr(precipitation_example_site_entire, cdf_ds, reference_window)

#### 5. Adjust SPI values with zero precipitation probability.

The CDF adjusted for months of zero precipitation is once again transformed to the z-normal distribution:

\begin{equation}
\text{Adjusted SPI-index} = \Phi^{-1}\big(F_{\bar{p}_{0}}(x,  \alpha, \beta, \lambda)\big)
\end{equation}

In [ ]:
def cdf_to_spi_transform(adjusted_cdf_ds):

    clipped = adjusted_cdf_ds.clip(1e-16, 1 - 1e-16)
    
    # Convert CDF to index
    adjusted_spi_ds = xr.apply_ufunc(stats.norm.ppf, clipped, 0.0, 1.0,  
                           input_core_dims=[[], [], []], output_core_dims=[[]],
                           vectorize=True, dask="parallelized",
                           output_dtypes=[np.float64],
                           keep_attrs=True
    )
    
    accumulation_variables = {var: var.split("_")[-1][:-1] for var in adjusted_spi_ds.variables if "accum" in var}  # Get periods as number strings
    rename_variables = {var: f"{"SPI"}{accumulation_period}" for var, accumulation_period in accumulation_variables.items()}
    
    return adjusted_spi_ds.rename_vars(rename_variables)

adjusted_spi_ds = cdf_to_spi_transform(adjusted_cdf_ds)

#### Importing ERA5-Drought SPI data from dataset for comparison.
Data request must be "weaved" as request too big for entire time range.

In [ ]:
drought_dataset = "derived-drought-historical-monthly"

spi_request1 = {
    "variable": ["standardised_precipitation_index"],
    "accumulation_period": [
        "1",
        "3",
        "6",
        "12",
        "24",
        "36",
        "48"
    ],
    "version": "1_0",
    "product_type": ["reanalysis"],
    "dataset_type": "consolidated_dataset",
    "year": [
        "1940", "1941", "1942",
        "1943", "1944", "1945",
        "1946", "1947", "1948",
        "1949", "1950", "1951",
        "1952", "1953", "1954",
        "1955", "1956", "1957",
        "1958", "1959", "1960",
        "1961", "1962", "1963",
        "1964", "1965", "1966",
        "1967", "1968", "1969",
        "1970", "1971", "1972",
        "1973", "1974", "1975",
        "1976", "1977", "1978",
        "1979", "1980"
    ],
    "month": [
        "01", "02", "03",
        "04", "05", "06",
        "07", "08", "09",
        "10", "11", "12"
    ],
    "area": [9.45, 40.25, 8.95, 40.75]
}

spi_request2 = {
    "variable": ["standardised_precipitation_index"],
    "accumulation_period": [
        "1",
        "3",
        "6",
        "12",
        "24",
        "36",
        "48"
    ],
    "version": "1_0",
    "product_type": ["reanalysis"],
    "dataset_type": "consolidated_dataset",
    "year": [
        "1981", "1982", "1983",
        "1984", "1985", "1986",
        "1987", "1988", "1989",
        "1990", "1991", "1992",
        "1993", "1994", "1995",
        "1996", "1997", "1998",
        "1999", "2000", "2001",
        "2002", "2003", "2004",
        "2005", "2006", "2007",
        "2008", "2009", "2010",
        "2011", "2012", "2013",
        "2014", "2015", "2016",
        "2017", "2018", "2019",
        "2020"
    ],
    "month": [
        "01", "02", "03",
        "04", "05", "06",
        "07", "08", "09",
        "10", "11", "12"
    ],
    "area": [9.45, 40.25, 8.95, 40.75] # Ethiopia
}

era5_drought_spi = ekd.from_source("cds", drought_dataset, spi_request1, spi_request2) # Sends request for this dataset to CDS.
era5_drought_spi = era5_drought_spi.to_xarray(compat="equals") # Converts to xarray.
era5_drought_spi = era5_drought_spi.sel(lat=9.25,lon=40.5, method="nearest")

#### Comparison of the calculated SPI-index vs ERA5-Drought SPI-index (qualatitive)
Now that we have calculated the SPI-index for one grid point, for all accumulation windows, we make a qualatitive and quantitative comparison with the corresponding data in the ERA5-Drought dataset. 

First, we plot the timeseries of the SPI drought index, calculated and from the derived dataset.

In [ ]:
# Create a 4x4 grid of subplots
fig, axs = plt.subplots(4, 2, figsize=(16, 12), sharex=True, sharey=True, constrained_layout=True)
fig.suptitle('ERA5-Drought SPI vs Calculated SPI for varying windows.', fontsize=16)

# Flatten axs for easy indexing
axs = axs.flatten()

for position, period in enumerate(ACCUMULATION_PERIODS):
    ax = axs[position]

    # Plot ERA5 drought SPI
    ax.plot(era5_drought_spi[f"SPI{period}"].time,
            era5_drought_spi[f"SPI{period}"],
            label=f"ERA5_Drought-SPI{period}",
            color="tab:blue")

    # Plot calculated SPI
    ax.plot(adjusted_spi_ds[f"SPI{period}"].valid_time,
            adjusted_spi_ds[f"SPI{period}"],
            label=f"Calculated SPI{period}",
            color="tab:orange")

    # Title for each subplot
    ax.sharex(axs[0])
    ax.set_title(f"{period}-Month Window", fontsize=10)
    ax.set_xlabel('Date')
    ax.set_ylabel('SPI')    
    ax.grid(True)

# Remove unused axes
for i in range(len(ACCUMULATION_PERIODS), len(axs)):
    fig.delaxes(axs[i])

# Add a single legend for the whole figure
handles, labels = axs[0].get_legend_handles_labels()
fig.legend(handles, labels, loc="upper left", ncol=2, frameon=True)
plt.show()

#### Comparison of the calculated SPI-index vs ERA5-Drought SPI-index (quantative- plot of residuals & frequency histogram).


In [ ]:
residual  = adjusted_spi_ds_focus[f"SPI1"] - era5_drought_spi[f"SPI1"]

In [ ]:
adjusted_spi_ds_focus = adjusted_spi_ds.rename({
    "valid_time": "time",
    "latitude": "lat",
    "longitude": "lon",
}) # for alignment

# Create a 4x4 grid of subplots
fig, axs = plt.subplots(4, 2, figsize=(16, 12), sharex=True, sharey=False, constrained_layout=True)
fig.suptitle('Residual SPI for varying windows.', fontsize=16)

# Flatten axs for easy indexing
axs = axs.flatten()

for position, period in enumerate(ACCUMULATION_PERIODS):
    ax = axs[position]
    # Plot residual between ERA5-SPI & Calculated-SPI
    ax.plot(residual.time,
            residual,
            label=f"Residual-SPI{period}",
            color="tab:blue")

    # Title for each subplot
    ax.sharex(axs[0])
    ax.set_title(f"{period}-Month Window", fontsize=10)
    ax.set_xlabel('Date')
    ax.set_ylabel('Residual SPI')    
    ax.grid(True)
    # ax.set_ylim([-1e-5, 1e-5])

# Remove unused axes
for i in range(len(ACCUMULATION_PERIODS), len(axs)):
    fig.delaxes(axs[i])

# Add a single legend for the whole figure
handles, labels = axs[0].get_legend_handles_labels()
fig.legend(handles, labels, loc="upper left", ncol=2, frameon=True)
plt.show()

In [ ]:
adjusted_spi_ds_focus = adjusted_spi_ds.rename({
    "valid_time": "time",
    "latitude": "lat",
    "longitude": "lon",
}) # for alignment

# Create a 4x4 grid of subplots
fig, axs = plt.subplots(4, 2, figsize=(16, 12), sharex=False, sharey=False, constrained_layout=True)
fig.suptitle('Residual SPI for varying windows.', fontsize=16)

# Flatten axs for easy indexing
axs = axs.flatten()

for position, period in enumerate(accum_periods):
    ax = axs[position]
    
    residual  = adjusted_spi_ds_focus[f"SPI_{period}m"] - era5_drought_spi[f"SPI{period}"]
    
    vals = (residual.values).ravel() 
    
    finite_mask = np.isfinite(vals)
    
    vals = vals[finite_mask]

    count = vals.size
    mean_val = np.nanmean(vals)
    std_val = np.nanstd(vals)

    freq, bin_edges = np.histogram(vals, bins=20)
    rel_freq = freq / count
    bin_centers = (bin_edges[:-1] + bin_edges[1:]) / 2

    # Plot relative frequency histogram
    ax.bar(bin_centers, rel_freq, width=(bin_edges[1] - bin_edges[0]))
    
    ax.annotate(
            f"Mean residual - SPI {period}: {mean_val: .7f}\n Std residual- SPI {period}: {std_val: .7f}",
            xy=(0.02, 0.95), xycoords="axes fraction",  # position in axes coords
            fontsize=10,
            ha="left", va="top",
        )
    
    # Title for each subplot
    ax.sharex(axs[0])
    ax.set_title(f"{period}-Month Window", fontsize=10)
    ax.set_xlabel('Residual SPI')
    ax.set_ylabel('Frequency')    
    ax.grid(True)
    
# Remove unused axes
for i in range(len(accum_periods), len(axs)):
    fig.delaxes(axs[i])

# Add a single legend for the whole figure
handles, labels = axs[0].get_legend_handles_labels()
fig.legend(handles, labels, loc="upper left", ncol=2, frameon=True)
plt.show()

####  6. Comparison of Probability of Zero Precipitation to that from ERA5-Drought dataset.
You can also import the "probability of zero precipitation" for a given calendar month, in the reference period, per accumulation period instead of calculating it as we have done. 

Due to improper formatting from ERA5-Drought, you are unable to send an API request to download the data for all accumulation periods at once. If you try to do so, you will find that your xarray only contains the data for the last accumulation period (in this case, the 48-month window). 

We have written a simple helper function, "era5_api_multiple" that sends separate API requests, concatenating the data into a single xarray, to then use.

In [ ]:
prob_spi_zero_all = era5_drought_api_multiple(indicator = "spi", var = "prob_zero")
prob_spi_zero_all_ds = prob_spi_zero_all.sel(lat=9.25,lon=40.5, method="nearest")

In [ ]:
prob_zero_diff = prob_spi_zero_all_ds.groupby('time.month') - stats_sum["Zero-Precip Count"]
cond =  (prob_zero_diff != 0).compute()
mis_match = prob_zero_diff.where(cond, drop=True)
mis_match

#### 7. Quality control using the Shapiro-Wilks test on calculated SPI data.
Quality control is performed by the ERA5-Drought team over the entire dataset. This is done by testing if the calculated distribution of the estimated drought indices over the reference period follows a normal distribution with mean 0 and standard deviation 1. 

This test is performed using the Shapiro-Wilks test for normality [S. S. SHAPIRO, M. B. WILK, An analysis of variance test for normality (complete samples), Biometrika, Volume 52, Issue 3-4, December 1965, Pages 591–611](https://doi.org/10.1093/biomet/52.3-4.591), with a $\alpha$ = 0.05 on the data in the reference period (1991-2020). 

If the resultant p-value is less than $\alpha$ = 0.05, the corresponding quality parameter is set to 0 (bad), otherwise set to 1 (good).

##### We perform this test over the calculated SPI values, that are adjusted for zero-precipitation. 

In [ ]:
stat, pval, sig = xr_shapiro_test(adjusted_spi_ds)

##### Importing Shapiro-Wilks SPI Significance Test results from ERA5-Drought dataset.
You can also import the quality flags from ERA5-Drought, for every calendar month, in the reference period, per accumulation period. 

Due to improper formatting from ERA5-Drought, you are unable to send an API request to download this data for all accumulation periods at once. If you try to do so, you will find that your xarray only contains the data for the last accumulation period (in this case, the 48-month window). 

We have written a simple script that sends separate API requests, concatenating the data into a single xarray, to then use.

In [ ]:
quality_spi_all = era5_drought_api_multiple(indicator = "spi", var = "quality")
quality_spi_all = quality_spi_all.sel(lat=9.25, lon = 40.5)

##### Comparison of significance values

In [ ]:
sig_diff = quality_spi_all.groupby('time.month') - sig
cond =  (sig_diff != 0).compute()
# Does cond have any True anywhere?
any_true = cond.any().compute()   # -> bool

if not any_true:
    print("Values from ERA5 Significance match calculated significance values.")

#### Calculating ERA5-Drought SPI (across a region)
Repeat calculations here but very quickly (i.e. just follow the steps from before but without all the steps and plots in between)
All functions should be written so that they can take data with only one (lat, lon) point as before or the entire dataset at once

##### Accumulate precipitations

In [ ]:
acc_periods = [48]

# Select latitude and longitude box (keeping small for smaller compute time, recommended 10 x 10)
lon_min, lon_max = 19.0, 21.5   
lat_min, lat_max = 41.0, 43.5

# Subset (latitude is descending in ERA5)
ds_loc = era5_monthly_mean_reanal.sel(
    latitude=slice(lat_max, lat_min),
    longitude=slice(lon_min, lon_max),
)

ds_loc = accum_var(ds_loc, var = "tp")

precipitation_example_site_reference = ds_loc.sel(**reference_window) # select only reference precipitation window.

##### Fit gamma distribution onto site.

In [ ]:
params_fitted = fit_monthly_spi(precipitation_example_site_reference)

##### Calculate SPI over region.

In [ ]:
precipitation_example_site_entire = ds_loc.sel(**entire_window) # select only reference precipitation window.
cdf_ds, spi_ds = compute_monthly_spi(precipitation_example_site_entire, params_fitted)

##### Adjust for zero precipitation

In [ ]:
adjusted_cdf_ds, stats_sum = zero_precip_monthly_xr(precipitation_example_site_entire, cdf_ds, reference_window) # takes a while

In [ ]:
adjusted_spi_ds = cdf_to_spi_transform(adjusted_cdf_ds)

In [ ]:
dataset = "derived-drought-historical-monthly"

request1 = {
    "variable": ["standardised_precipitation_index"],
    "accumulation_period": [
        "48"
    ],
    "version": "1_0",
    "product_type": ["reanalysis"],
    "dataset_type": "consolidated_dataset",
    "year": [
        "1940", "1941", "1942",
        "1943", "1944", "1945",
        "1946", "1947", "1948",
        "1949", "1950", "1951",
        "1952", "1953", "1954",
        "1955", "1956", "1957",
        "1958", "1959", "1960",
        "1961", "1962", "1963",
        "1964", "1965", "1966",
        "1967", "1968", "1969",
        "1970", "1971", "1972",
        "1973", "1974", "1975",
        "1976", "1977", "1978",
        "1979", "1980"
    ],
    "month": [
        "01", "02", "03",
        "04", "05", "06",
        "07", "08", "09",
        "10", "11", "12"
    ], 
    "area": [43.5, 19.0, 41.0, 21.5]  # lat, lon 
}

request2 = {
    "variable": ["standardised_precipitation_index"],
    "accumulation_period": [
        "48"
    ],
    "version": "1_0",
    "product_type": ["reanalysis"],
    "dataset_type": "consolidated_dataset",
    "year": [
        "1981", "1982", "1983",
        "1984", "1985", "1986",
        "1987", "1988", "1989",
        "1990", "1991", "1992",
        "1993", "1994", "1995",
        "1996", "1997", "1998",
        "1999", "2000", "2001",
        "2002", "2003", "2004",
        "2005", "2006", "2007",
        "2008", "2009", "2010",
        "2011", "2012", "2013",
        "2014", "2015", "2016",
        "2017", "2018", "2019",
        "2020"
    ],
    "month": [
        "01", "02", "03",
        "04", "05", "06",
        "07", "08", "09",
        "10", "11", "12"
    ],
    "area": [43.5, 19.0, 41.0, 21.5] # lat, lon 
}

era5_spi = ekd.from_source("cds", dataset, request1,request2) # Sends request for this dataset to CDS.
era5_spi = era5_spi.to_xarray(compat="equals") # Converts to xarray.

In [ ]:
adjusted_spi_ds = adjusted_spi_ds.rename({
    "valid_time": "time",
    "latitude": "lat",
    "longitude": "lon"})

In [ ]:
spi_drought, adjusted_spi_ds = xr.align(spi_drought, adjusted_spi_ds, join="left")   # only overlapping coords
# spi_diff_mean = spi_diff.mean(dim="time")

In [ ]:
spi_diff_mean = abs(adjusted_spi_ds["SPI48"]-spi_drought["SPI48"]).mean("time")

In [ ]:
spi_diff = abs(adjusted_spi_ds["SPI48"]-spi_drought["SPI48"])

In [ ]:
# Select one time slice and one ensemble
from earthkit.plots.styles import Style
from earthkit.plots.geo import domains

global_mean_map = spi_diff

balkans = domains.Domain.from_bbox(
    bbox=[15, 25, 36, 47],
    name="Balkans",
)
# Convert to NumPy arrays

mean_values = global_mean_map.to_numpy()
lat_values = global_mean_map.lat.to_numpy()
lon_values = global_mean_map.lon.to_numpy()
 
# Create meshgrid

lon_grid, lat_grid = np.meshgrid(lon_values, lat_values)
 
# # Plot with EarthKit

SPI_STYLE = Style(cmap='viridis', vmin = -0.1, vmax = 0.1, normalize=False)

# Create figure with 2 columns
fig = ekp.Figure(rows=1, columns=1, size=(12, 6))  

# First subplot (left)
subplot = fig.add_map(domain=balkans, row=0, column=0)
subplot.grid_cells(mean_values, x=lon_grid, y=lat_grid,style=SPI_STYLE)
subplot.legend(location="right")

# Add decorations
fig.title("ERA5 - Calculated SPI")
fig.land()
fig.coastlines()
fig.borders()
fig.gridlines()
fig.show()

(section-3)=
###  3. Calculating ERA5-Drought SPEI from reanalysis data.
The steps to calculating the SPEI-index are exactly the same as calculating the SPI index, with the only modification being that the SPEI integrates both the precipitation and potential evapotranspiration (PET) data. 

#### 0. Importing monthly-average potential evaporation data.

We import the monthly-mean potential evaporation data from the "ERA5 monthly-averaged data on single levels from 1940 to present" dataset.(https://cds.climate.copernicus.eu/datasets/reanalysis-era5-single-levels-monthly-means?tab=download)

In [ ]:
request_era5_pev_moda = {
    "param": "251.228",       # Variable: Potential evaporation (pev)
    "stream": "moda",         # Data stream: Monthly mean reanalysis
} | request_era5_template

In [ ]:
# Download data and convert to desired format
era5_monthly_mean_pev_reanal = ekd.from_source("cds", ID_ERA5, request_era5_pev_moda)  # Download as field list
era5_monthly_mean_pev_reanal = era5_monthly_mean_pev_reanal.to_xarray(compat="equals")  # Convert to xarray dataset
era5_monthly_mean_pev_reanal = rechunk(era5_monthly_mean_pev_reanal)  # Re-chunk for speed gain in fitting
era5_monthly_mean_pev_reanal  # Display in notebook

#### 1. Calculate moving average for different accumulation periods (PET/PEV & TP).

A time series of both precipitation & potential evaporation (PET/PEV) from one grid point are extracted and the precipitation & potential evaporation are accumulated over the previous $n$ months using a moving window, analogous to the SPI.

Note, [ECMWF convention](https://codes.ecmwf.int/grib/param-db/182) is that negative values for PEV/PET indicate evaporation, whereas positive values indicate condensation.

In [ ]:
# Select the grid point and time slice for Ethiopia
pev_example_site = era5_monthly_mean_pev_reanal.sel(**example_site)

# Perform the accumulation for each accumulation period
pev_example_site = accum_var(pev_example_site, "pev")

# Display result
pev_example_site

plt.figure(figsize=(12, 6))
for p in ACCUMULATION_PERIODS:
    var_name = f'pev_mm_accum_{p}m'
    plt.plot(pev_example_site['valid_time'].values, pev_example_site[var_name].values, label=f'{p}-month')

# Customize plot
plt.title('Potential Evaporation Accumulation at (9.25°N, 40.5°E)')
plt.xlabel('Time')
plt.ylabel('PEV (mm)')
plt.legend(title='Accumulation Period')
plt.grid(True)
plt.tight_layout()
plt.show()

In [ ]:
wb_ds = xr.Dataset(
    {
        f"wb_{var.replace('tp_', '')}": precipitation_example_site[var]
                                       + pev_example_site[var.replace("tp_", "pev_")]
        for var in precipitation_example_site.data_vars # loop through every variable.
        if var.startswith("tp_") and var.replace("tp_", "pev_") in pev_example_site.data_vars
    }
)

#### Plot of the accumulated water balance (P-PET).
Since the ECMWF convention is that a negative value of PET indicates evaporation, care must be taken when subtracting the precipitation by the potential evaporation (PEV/PET).

The negative of PEV/PET must be applied, meaning that $P − (−PET) = P + PET$ in this case. 

This approach aligns with the definition of the water balance, where a negative water balance value indicates that more water is potentially being transferred to the atmosphere. It also makes sense when interpreting the accumulated plots of both potential evaporation and water balance in a country like Ethiopia (below), where one might expect more evaporation and in fact has a negative water balance (as is the case).



In [ ]:
plt.figure(figsize=(12, 6))
for p in ACCUMULATION_PERIODS:
    var_name = f'wb_mm_accum_{p}m'
    plt.plot(wb_ds['valid_time'].values, wb_ds[var_name].values, label=f'{p}-month')

# Customize plot
plt.title('Total Precipitation - (- Potential Evaporation) at (9.25°N, 40.5°E)')
plt.xlabel('Time')
plt.ylabel('TP - PEV (mm)')
plt.legend(title='Accumulation Period')
plt.grid(True)
plt.tight_layout()
plt.show()

#### 2. Fit Generalised Log-Logistic Distribution to PET.

The general log-logistic distribution [reference] is fitted only to the data within the reference period (1991-2020), similar to the gamma distribution being fitted in the calculation of the SPI-index. A separate distribution is fitted for each calendar month per accumulation window.

This fitting is done with the [scipy.stats.genlogistic](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.genlogistic.html) object in Python. Three parameters are then outputted per calendar month, per accumulation period: the shape (), location () and scale () parameter.

In [ ]:
# Fit log logistic distributions
log_params_fitted = fit_monthly_spei(wb_ds.sel(**reference_window))

#### 3. Compute SPEI series.

Similar to the calculation for the SPI index, the three parameters from the fitted general log-logistic distribution are taken and the cumulative distribution function (CDF) is calculated up to the accumulated water balance, from the probability distribution function (PDF).

As $ P − PET $ is barely identical to 0, no modifications analogous to the SPI such as adjusting for zero precipitation, are required.

In [ ]:
params_fitted = log_params_fitted.rename({v: v.replace("tp_", "wb_")
                                          for v in log_params_fitted.data_vars})

In [ ]:
# Compute SPI series
cdf_spei_ds, spei_ds = compute_monthly_spei(wb_ds, params_fitted)

#### 4. Quality control using the Shapiro-Wilks test on calculated SPEI data.

Shapiro-Wilks test is performed on the calculated SPEI index below.

In [ ]:
stat, pval, sig = xr_shapiro_test(spei_ds.sel(**reference_window))

##### Importing Shapiro-Wilks SPI Significance Test results from ERA5-Drought dataset.
Quality flags ar imported from ERA5-Drought, for every calendar month, in the reference period, per accumulation period for the SPEI index.

Due to improper formatting from ERA5-Drought, you are unable to send an API request to download this data for all accumulation periods at once. If you try to do so, you will find that your xarray only contains the data for the last accumulation period (in this case, the 48-month window). 

We have written a simple script that sends separate API requests, concatenating the data into a single xarray, to then use.

In [ ]:
quality_spei_all = era5_drought_api_multiple(indicator = "spei", var = "quality")

##### Comparing Calculated Shapiro-Wilks SPI Significance Test with that from ERA5-Drought dataset.

In [ ]:
# select quality flags at one location and compare that with dataset.
quality_spei_all_ds = quality_spei_all.sel(lat=9.25,lon=40.5, method="nearest")

sig_diff = quality_spei_all_ds.groupby('time.month') - sig
cond =  (sig_diff != 0).compute()
# Does cond have any True anywhere?
any_true = cond.any().compute()   # -> bool

if not any_true:
    print("Values from ERA5 Significance match calculated significance values.")

#### Comparison of the calculated SPEI-index vs ERA5-Drought SPEI-index (qualatitive)

In [ ]:
dataset = "derived-drought-historical-monthly"
request1 = {
    "variable": ["standardised_precipitation_evapotranspiration_index"],
    "accumulation_period": [
        "1",
        "3",
        "6",
        "12",
        "24",
        "36",
        "48"
    ],
    "version": "1_0",
    "product_type": ["reanalysis"],
    "dataset_type": "consolidated_dataset",
    "year": [
        "1940", "1941", "1942",
        "1943", "1944", "1945",
        "1946", "1947", "1948",
        "1949", "1950", "1951",
        "1952", "1953", "1954",
        "1955", "1956", "1957",
        "1958", "1959", "1960",
        "1961", "1962", "1963",
        "1964", "1965", "1966",
        "1967", "1968", "1969",
        "1970", "1971", "1972",
        "1973", "1974", "1975",
        "1976", "1977", "1978",
        "1979", "1980", "1981"
    ],
    "month": [
        "01", "02", "03",
        "04", "05", "06",
        "07", "08", "09",
        "10", "11", "12"
    ],
    "area": [9.45, 40.25, 8.95, 40.75]
}

request2 = {
    "variable": ["standardised_precipitation_evapotranspiration_index"],
    "accumulation_period": [
        "1",
        "3",
        "6",
        "12",
        "24",
        "36",
        "48"
    ],
    "version": "1_0",
    "product_type": ["reanalysis"],
    "dataset_type": "consolidated_dataset",
    "year": [
        "1982", "1983", "1984",
        "1985", "1986", "1987",
        "1988", "1989", "1990",
        "1991", "1992", "1993",
        "1994", "1995", "1996",
        "1997", "1998", "1999",
        "2000", "2001", "2002",
        "2003", "2004", "2005",
        "2006", "2007", "2008",
        "2009", "2010", "2011",
        "2012", "2013", "2014",
        "2015", "2016", "2017",
        "2018", "2019", "2020"
    ],
    "month": [
        "01", "02", "03",
        "04", "05", "06",
        "07", "08", "09",
        "10", "11", "12"
    ],
    "area": [9.45, 40.25, 8.95, 40.75]
}

data_spei = ekd.from_source("cds", dataset, request1,request2) # Sends request for this dataset to CDS.
data_spei = data_spei.to_xarray(compat="equals") # Converts to xarray.
data_spei = data_spei.sel(lat=9.25,lon=40.5, method="nearest")

In [ ]:
# Create a 4x4 grid of subplots
fig, axs = plt.subplots(4, 2, figsize=(16, 12), sharex=True, sharey=True, constrained_layout=True)
fig.suptitle('ERA5-Drought SPEI vs Calculated SPEI for varying windows.', fontsize=16)

data_spei_focus, spei_ds_focus = xr.align(data_spei, spei_ds, join = "override")   # only overlapping coords

# Flatten axs for easy indexing
axs = axs.flatten()

for position, period in enumerate(ACCUMULATION_PERIODS):
    ax = axs[position]

    # Plot ERA5 drought SPI
    ax.plot(data_spei_focus[f"SPEI{period}"].time,
            data_spei_focus[f"SPEI{period}"],
            label=f"ERA5_Drought-SPEI{period}",
            color="tab:blue")

    # Plot calculated SPI
    ax.plot(spei_ds[f"SPEI{period}"].valid_time,
            spei_ds[f"SPEI{period}"],
            label=f"Calculated SPEI{period}",
            color="tab:orange")

    # Title for each subplot
    ax.sharex(axs[0])
    ax.set_title(f"{period}-Month Window", fontsize=10)
    ax.set_xlabel('Date')
    ax.set_ylabel('SPEI')    
    ax.grid(True)

# Remove unused axes
for i in range(len(accum_periods), len(axs)):
    fig.delaxes(axs[i])

# Add a single legend for the whole figure
handles, labels = axs[0].get_legend_handles_labels()
fig.legend(handles, labels, loc="upper left", ncol=2, frameon=True)
plt.show()

#### Comparison of the calculated SPEI-index vs ERA5-Drought SPEI-index (quantatitive)

In [ ]:
accum_periods = [1, 3, 6, 12, 24, 36, 48]

spei_ds_focus = spei_ds.rename({
    "valid_time": "time",
    "latitude": "lat",
    "longitude": "lon",
}) # for alignment

# Create a 4x4 grid of subplots
fig, axs = plt.subplots(4, 2, figsize=(16, 12), sharex=True, sharey=False, constrained_layout=True)
fig.suptitle('Residual SPEI for varying windows.', fontsize=16)

# Flatten axs for easy indexing
axs = axs.flatten()

for position, period in enumerate(accum_periods):
    ax = axs[position]
    residual  = spei_ds_focus[f"SPEI{period}"] - data_spei_focus[f"SPEI{period}"]
    # Plot residual between ERA5-SPI & Calculated-SPI
    ax.plot(residual.time,
            residual,
            label=f"Residual-SPEI",
            color="tab:blue")

    # Title for each subplot
    ax.sharex(axs[0])
    ax.set_title(f"{period}-Month Window", fontsize=10)
    ax.set_xlabel('Date')
    ax.set_ylabel('Residual SPEI')    
    ax.grid(True)
    # ax.set_ylim([-1e-5, 1e-5])

# Remove unused axes
for i in range(len(accum_periods), len(axs)):
    fig.delaxes(axs[i])

# Add a single legend for the whole figure
handles, labels = axs[0].get_legend_handles_labels()
fig.legend(handles, labels, loc="upper left", ncol=2, frameon=True)
plt.show()

#### Comparison of the calculated SPEI-index vs ERA5-Drought SPEI-index with quality flags (quantatitive)

In [ ]:
accum_periods = [1, 3, 6, 12, 24, 36, 48]

for period in accum_periods:
    drought_spi = data_spei_focus[f"SPEI{period}"].copy()
    calc_spi = spei_ds_focus[f"SPEI{period}"].copy() # copy so it doesn't change.
    
    for month in range(1,13):        
        significance = quality_spei_all_ds[f"significance_{period}"].sel(time=f"2020-{month:02d}-01").compute().item()
        if significance == 0:
            drought_spi = drought_spi.where(drought_spi.time.dt.month != month, other=np.nan) # keeps EVERY other month- the selected one becomes nan. 
            calc_spi = calc_spi.where(calc_spi.time.dt.month != month, other=np.nan)
        else:
            pass 
    
    data_spei_focus[f"SPEI{period}"] = drought_spi # bring it back.
    adjusted_spi_ds[f"SPEI{period}"] = calc_spi
    

In [ ]:
accum_periods = [1, 3, 6, 12, 24, 36, 48]

# data_spei_focus, spei_ds_focus = xr.align(data_spei_focus, spei_ds_focus, join = "override")   # only overlapping coords

# Create a 4x4 grid of subplots
fig, axs = plt.subplots(4, 2, figsize=(16, 12), sharex=True, sharey=False, constrained_layout=True)
fig.suptitle('Residual SPEI for varying windows w/ quality flags.', fontsize=16)

# Flatten axs for easy indexing
axs = axs.flatten()

for position, period in enumerate(accum_periods):
    ax = axs[position]
    residual  = adjusted_spi_ds[f"SPEI{period}"] - data_spei_focus[f"SPEI{period}"]
    # Plot residual between ERA5-SPI & Calculated-SPI
    ax.plot(residual.time,
            residual,
            label=f"Residual-SPEI",
            color="tab:blue")

    # Title for each subplot
    ax.sharex(axs[0])
    ax.set_title(f"{period}-Month Window", fontsize=10)
    ax.set_xlabel('Date')
    ax.set_ylabel('Residual SPEI')    
    ax.grid(True)
    # ax.set_ylim([-1e-5, 1e-5])

# Remove unused axes
for i in range(len(accum_periods), len(axs)):
    fig.delaxes(axs[i])

# Add a single legend for the whole figure
handles, labels = axs[0].get_legend_handles_labels()
fig.legend(handles, labels, loc="upper left", ncol=2, frameon=True)
plt.show()

(section-4)=
### 4. Calculating the SPI/SPEI Ensemble Data

#### Reading in ensemble precipitation 

In [ ]:
ID_ERA5 = "reanalysis-era5-complete"

request_era5_template = {
    "class": "ea",            # Default for ERA5
    # Dates: ERA5 takes these in the format 19400101/19400201/.../20241101/20241201
    # The following line generates a string in said format from the chosen year range
    "date": "/".join(f"{year}{month:02}01"
            for year in range(years[0] ,years[1]+1)
            for month in MONTHS),
    "expver": "1",            # ERA5 consolidated data
    "levtype": "sfc",         # Surface
    "grid": "0.25/0.25",      # Grid: 0.25° by 0.25°
    "type": "fc",             # Forecast
    "data_format": "netcdf",  # NetCDF data
}

In [ ]:
request_era5_precipitation_edmo = {
    "param": "228.128",       # Variable: Total precipitation
    "stream": "edmo",         # Data stream: Monthly mean reanalysis
} | request_era5_template

In [ ]:
request_era5_precipitation_edmo

In [ ]:
# Download data and convert to desired format
era5_monthly_mean_ens = ekd.from_source("cds", ID_ERA5, request_era5_precipitation_edmo)  # Download as field list
era5_monthly_mean_ens = era5_monthly_mean_ens.to_xarray(compat="equals")  # Convert to xarray dataset
era5_tp_monthly_mean_ens = rechunk(era5_monthly_mean_ens)  # Re-chunk for speed gain in fitting
era5_tp_monthly_mean_ens  # Display in notebook

#### Reading in ensemble potential evaporation 

In [ ]:
request_era5_pev_edmo = {
    "param": "251.228",       # Variable: Potential evaporation (pev)
    "stream": "edmo",         # Data stream: Monthly mean reanalysis
} | request_era5_template

In [ ]:
# Download data and convert to desired format
era5_pev_monthly_mean_ens = ekd.from_source("cds", ID_ERA5, request_era5_pev_edmo)  # Download as field list
era5_pev_monthly_mean_ens = era5_pev_monthly_mean_ens.to_xarray(compat="equals")  # Convert to xarray dataset
era5_pev_monthly_mean_ens = rechunk(era5_pev_monthly_mean_ens)  # Re-chunk for speed gain in fitting
era5_pev_monthly_mean_ens  # Display in notebook

#### Accumulate precipitation.

In [ ]:
example_site = {"latitude": 9.25, "longitude": 40.5,}

In [ ]:
# Select the grid point and time slice for Ethiopia
precipitation_example_site = era5_tp_monthly_mean_ens.sel(**example_site)

# Perform the accumulation for each accumulation period
precipitation_ens_example_site = accum_var(precipitation_example_site, "tp")

# Display result
precipitation_ens_example_site

#### Accumulate potential evaporation.

In [ ]:
# Select the grid point and time slice for Ethiopia
pev_example_site = era5_pev_monthly_mean_ens.sel(**example_site)

# Perform the accumulation for each accumulation period
pev_ens_example_site = accum_var(pev_example_site, "pev")

# Display result
pev_ens_example_site

#### Calculate the water balance (P - PET).

In [ ]:
wb_ens_example_site = xr.Dataset(
    {
        f"wb_{var.replace('tp_', '')}": precipitation_ens_example_site[var]
                                       + pev_ens_example_site[var.replace("tp_", "pev_")]
        for var in precipitation_ens_example_site.data_vars # loop through every variable.
        if var.startswith("tp_") and var.replace("tp_", "pev_") in pev_ens_example_site.data_vars
    }
)

In [ ]:
wb_ens_example_site

#### Fit gamma distribution to ensemble precipitation.

In [ ]:
# Fit log logistic distributions
gamma_ens_params_fitted = fit_monthly_spi(precipitation_ens_example_site.sel(**reference_window))

In [ ]:
gamma_ens_params_fitted

#### Fit log-logistic distribution to water balance.

In [ ]:
log_logistic_ens_params_fitted = fit_monthly_spei(wb_ens_example_site.sel(**reference_window))

In [ ]:
log_logistic_ens_params_fitted

#### Calculate SPI ensemble

In [ ]:
# Compute SPI series -- new function
cdf_ens_ds, spi_ens_ds = compute_monthly_spi(precipitation_ens_example_site, gamma_ens_params_fitted)

#### Calculate SPEI ensemble

In [ ]:
# Compute SPEI series -- new function
cdf_wb_ens_ds, spei_ens_ds = compute_monthly_spei(wb_ens_example_site, log_logistic_ens_params_fitted)

#### Quality control using the Shapiro-Wilks test on calculated ensemble SPI data.

In [ ]:
ens_spi_stat, ens_spi_pval, ens_spi_sig = xr_shapiro_test(spi_ens_ds.sel(**reference_window))

In [ ]:
dataset = "derived-drought-historical-monthly"
request = {
    "variable": ["test_for_normality_spi"],
    "accumulation_period": [
        "1",
        "3",
        "6",
        "12",
        "24",
        "36",
        "48"
    ],
    "version": "1_0",
    "product_type": ["ensemble_members"],
    "dataset_type": "consolidated_dataset",
    "month": [
        "01", "02", "03",
        "04", "05", "06",
        "07", "08", "09",
        "10", "11", "12"
    ]
}


#### Quality control using the Shapiro-Wilks test on calculated ensemble SPEI data.

In [ ]:
ens_spei_stat, ens_spei_pval, ens_spei_sig = xr_shapiro_test(spei_ens_ds.sel(**reference_window))

#### Reading in ensemble SPI 

In [ ]:
spi_ens_dataset = "derived-drought-historical-monthly"
spi_ens_request = {
    "variable": ["standardised_precipitation_index"],
    "accumulation_period": ["1", "12", "48"], # 1 is also already downloaded.
    "version": "1_0",
    "product_type": ["ensemble_members"],
    "dataset_type": "consolidated_dataset",
    "year": [
        "1940", "1941", "1942",
        "1943", "1944", "1945",
        "1946", "1947", "1948",
        "1949", "1950", "1951",
        "1952", "1953", "1954",
        "1955", "1956", "1957",
        "1958", "1959", "1960",
        "1961", "1962", "1963",
        "1964", "1965", "1966",
        "1967", "1968", "1969",
        "1970", "1971", "1972",
        "1973", "1974", "1975",
        "1976", "1977", "1978",
        "1979", "1980", "1981",
        "1982", "1983", "1984",
        "1985", "1986", "1987",
        "1988", "1989", "1990",
        "1991", "1992", "1993",
        "1994", "1995", "1996",
        "1997", "1998", "1999",
        "2000", "2001", "2002",
        "2003", "2004", "2005",
        "2006", "2007", "2008",
        "2009", "2010", "2011",
        "2012", "2013", "2014",
        "2015", "2016", "2017",
        "2018", "2019", "2020",
        "2021", "2022", "2023",
        "2024", "2025"
    ],
    "month": [
        "01", "02", "03",
        "04", "05", "06",
        "07", "08", "09",
        "10", "11", "12"
    ],
    "area":[10, 40, 9, 41]
}

point_drought_ens_ = ekd.from_source("cds", spi_ens_dataset, spi_ens_request) # Sends request for this dataset to CDS.

In [ ]:
point_drought_ens_ = point_drought_ens_.to_xarray(
    compat="override",
    combine="nested",     # <- key: disable by-coords alignment
    concat_dim="time",    # <- key: explicit concat dimension
    coords="minimal",
    data_vars="minimal",
    join="override",      # avoids reindex conflicts
    parallel=False
)

point_drought_ens_ = point_drought_ens_.sel(lat = 9.25, lon = 40.5) # note there are duplicate timestamps. each one is a ensemble member.

In [ ]:
def create_ens(dataset, no_ens = 10):
    _, index = np.unique(dataset['time'], return_index=True)
    
    ens_dataset = []
    
    for i in range(1,no_ens+1):
        ens_member = dataset.isel(time = index + i)    
        ens_dataset.append(ens_member)
        
    drought_ens = xr.concat(ens_dataset, dim="number")
    drought_ens = drought_ens.assign_coords(number=np.arange(1,11))  # or 1..10 if you prefer

    return drought_ens
    
spi_ens = create_ens(point_drought_ens_)

In [ ]:
spi_ens

In [ ]:
spi_ens_ds

In [ ]:
spi_ens_ds = spi_ens_ds.assign_coords({"number": (spi_ens_ds.number + 1)} ) 

In [ ]:
residual  = spi_ens_ds[f"SPI1"] - spi_ens[f"SPI1"]

In [ ]:
spi_ens_ds["number"].values

#### Time-series comparison of SPI at one location 

In [ ]:
# Ensure both datasets use the same time coordinate name
# (Choose either to rename; here I rename spi_ens_ds.valid_time -> time)
# spi_ens_ds = spi_ens_ds.rename({"valid_time": "time"})


ACCUMULATION_PERIODS_focus = [1, 12, 48]

# Create a 4x2 grid of subplots
fig, axs = plt.subplots(4, 2, figsize=(16, 12), sharex=True, sharey=True, constrained_layout=True)
fig.suptitle('ERA5-Drought SPI-Ensemble vs Calculated SPI-Ensemble for varying windows.', fontsize=16)

# Flatten axs for easy indexing
axs = axs.flatten()

for m in range(1,5):
    calculated = spi_ens_ds.sel(number=m)
    era5 = spi_ens.sel(number=m)
    for position, period in enumerate(ACCUMULATION_PERIODS_focus):
        ax = axs[position]

        var = f"SPI{period}"  # minimal change: pick the variable for this window

        # Plot ERA5 drought SPI (DataArray)
        ax.plot(era5["time"].values,
                era5.values,
                label=f"ERA5_Drought-SPI{period}",
                color="tab:blue",
                alpha=0.8)

        # Plot calculated SPI (DataArray)
        ax.plot(calculated.values,
                calculated.values,
                label=f"Calculated SPI{period}",
                color="tab:orange",
                alpha=0.8)

        # Title for each subplot
        ax.set_title(f"{period}-Month Window", fontsize=10)
        ax.set_xlabel('Date')
        ax.set_ylabel('SPI')
        ax.grid(True)

# Remove unused axes
for i in range(len(accum_periods), len(axs)):
    fig.delaxes(axs[i])

# Add a single legend for the whole figure
handles, labels = axs[0].get_legend_handles_labels()
fig.legend(handles, labels, loc="upper left", ncol=2, frameon=True)
plt.show()

In [ ]:
ACCUMULATION_PERIODS_focus = [1, 12, 48]  # choose the periods you want

# Loop over each ensemble member and make one figure per member
for m in range(1, 11):  # 1..10 members
    calculated = spi_ens_ds.sel(number=m)
    era5 = spi_ens.sel(number=m)

    # Create a grid of subplots for the selected periods (here: 3 periods → 2x2 to leave one empty)
    nrows, ncols = 2, 2  # adjust if you change number of periods
    fig, axs = plt.subplots(nrows, ncols, figsize=(12, 8), sharex=True, sharey=True, constrained_layout=True)
    fig.suptitle(f'ERA5-Drought vs Calculated SPI — Member {m}', fontsize=16)

    axs = axs.flatten()

    for position, period in enumerate(ACCUMULATION_PERIODS_focus):
        ax = axs[position]
        var = f"SPI{period}"

        # Select the DataArray for this period and sort by time
        # (Assumes both have coord named "time"; otherwise rename once above)
        era5_da = era5[var].sortby("time")
        calc_da = calculated[var].sortby("time")

        # Plot ERA5
        ax.plot(
            era5_da["time"].values,
            era5_da.values,
            label=f"ERA5_Drought-SPI{period}",
            color="tab:blue",
            alpha=0.8,
        )

        # Plot Calculated
        ax.plot(
            calc_da["time"].values,
            calc_da.values,
            label=f"Calculated SPI{period}",
            color="tab:orange",
            alpha=0.8,
        )

        # Titles & axes
        ax.set_title(f"{period}-Month Window", fontsize=11)
        ax.set_xlabel("Date")
        ax.set_ylabel("SPI")
        ax.grid(True)

    # Remove any unused axes (e.g., if 3 periods in a 2x2 grid)
    for i in range(len(ACCUMULATION_PERIODS_focus), len(axs)):
        fig.delaxes(axs[i])

    # Build a single legend per figure (deduplicate handles)
    handles, labels = [], []
    for ax in axs[:len(ACCUMULATION_PERIODS_focus)]:
        h, l = ax.get_legend_handles_labels()
        for hh, ll in zip(h, l):
            if ll not in labels:
                handles.append(hh)
                labels.append(ll)

    fig.legend(handles, labels, loc="upper left", ncol=2, frameon=True)

    # Optionally save each member's figure
    # fig.savefig(f"spi_era5_vs_calc_member_{m}.png", dpi=200)

    plt.show()


#### Reading in ensemble SPEI

In [ ]:
spi_ens_dataset = "derived-drought-historical-monthly"
spi_ens_request = {
    "variable": ["standardised_precipitation_evaporation_index"],
    "accumulation_period": ["1", "12", "48"], # 1 is also already downloaded.
    "version": "1_0",
    "product_type": ["ensemble_members"],
    "dataset_type": "consolidated_dataset",
    "year": [
        "1940", "1941", "1942",
        "1943", "1944", "1945",
        "1946", "1947", "1948",
        "1949", "1950", "1951",
        "1952", "1953", "1954",
        "1955", "1956", "1957",
        "1958", "1959", "1960",
        "1961", "1962", "1963",
        "1964", "1965", "1966",
        "1967", "1968", "1969",
        "1970", "1971", "1972",
        "1973", "1974", "1975",
        "1976", "1977", "1978",
        "1979", "1980", "1981",
        "1982", "1983", "1984",
        "1985", "1986", "1987",
        "1988", "1989", "1990",
        "1991", "1992", "1993",
        "1994", "1995", "1996",
        "1997", "1998", "1999",
        "2000", "2001", "2002",
        "2003", "2004", "2005",
        "2006", "2007", "2008",
        "2009", "2010", "2011",
        "2012", "2013", "2014",
        "2015", "2016", "2017",
        "2018", "2019", "2020",
        "2021", "2022", "2023",
        "2024", "2025"
    ],
    "month": [
        "01", "02", "03",
        "04", "05", "06",
        "07", "08", "09",
        "10", "11", "12"
    ],
    "area":[10, 40, 9, 41]
}

point_drought_ens_ = ekd.from_source("cds", spi_ens_dataset, spi_ens_request) # Sends request for this dataset to CDS.
point_drought_ens_ = point_drought_ens_.to_xarray(compat="equals") # Converts to xarray.
point_drought_ens_ = point_drought_ens_.sel(lat = 9.25, lon = 40.5) # note there are duplicate timestamps. each one is a ensemble member.

#### Selecting drought index from ensemble at one location (lon=9.5, lat = 40.5)

In [ ]:
point_drought_ens_ = point_drought_ens_.sel(lon = 9.5, lat = 40.5)

In [ ]:
def make_ensemble(da, time_dim="time", member_dim="member"):
    times = pd.Index(da[time_dim].values)
    member_ids = pd.Series(times).groupby(times).cumcount().to_numpy()
    
    # Create MultiIndex
    mi = pd.MultiIndex.from_arrays([times, member_ids], names=[time_dim, member_dim])
    
    # Assign MultiIndex and rename the dimension to something temporary
    da = da.rename({time_dim: "tmp"}).assign_coords(tmp=mi)
    
    # Unstack to get (time, member)
    return da.unstack()

ens = make_ensemble(point_drought_ens_["SPI1"])  # or pass the whole Dataset

## ℹ️ If you want to know more

### Key resources
The CDS catalogue entries for the data used were:
* Complete ERA5 global atmospheric reanalysis: [reanalysis-era5-complete](https://doi.org/10.24381/cds.143582cf)
* Monthly drought indices from 1940 to present derived from ERA5 reanalysis: [derived-drought-historical-monthly](https://doi.org/10.24381/9bea5e16)

Code libraries used:
* [earthkit](https://github.com/ecmwf/earthkit)
  * [earthkit-data](https://github.com/ecmwf/earthkit-data)
  * [earthkit-plots](https://github.com/ecmwf/earthkit-plots)

### References

List the references used in the Notebook here.

E.g.

[[1]](https://doi.org/10.1038/s41598-018-20628-2) Rodriguez, D., De Voil, P., Hudson, D., Brown, J. N., Hayman, P., Marrou, H., & Meinke, H. (2018). Predicting optimum crop designs using crop models and seasonal climate forecasts. Scientific reports, 8(1), 2231.